In [1]:
#   CME Futures and Options settlements web page scraper and Options implied vol calculator
#   @author: Luis Molina, May 17, 2020 
#
#   Needs Firefox geckodriver and web browser to run
#   Needs python splinter library:  pip install splinter --user
#   For OLS needs statsmodels

import pandas as pd
import numpy as np
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.support.ui import Select

import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import scipy.stats as ss
import os
import logging
from datetime import datetime
from dateutil.parser import parse
import time
import xlwings

# debugging information
logging.basicConfig(level=logging.INFO)
import warnings
warnings.simplefilter('ignore')

import seaborn
seaborn.set()


ROOT_DIR = os.path.dirname(os.path.abspath('~/'))

In [2]:
class GetData:
    def __init__(self, tradeDate, futures_sym='NG', options_sym='LN', expiry_month='N', year='0'):
        
        self.futures_sym = futures_sym
        self.options_sym = options_sym
        self.expiry_month = expiry_month
        self.expiry_year = year
        self.rate = 0.0035
        self.basis = 365.
        self.carry = 0.      # for futures, generalized BlackScholes
        self.epsilon = 0.001 # precision for implied vol calc.
        self.tradeDate = tradeDate
        self.contract = f"{self.futures_sym}{self.expiry_month}2{self.expiry_year}"
        
        self._selbrowser = None
        self._splbrowser = None
    
        self.SRC_FILENAME = '{0}_options_{1}.pkl'.format(self.options_sym.lower(), self.tradeDate)
        self.SRC_VOL_FILE = '{0}_vol_surface_{1}.pkl'.format(self.options_sym.lower(), self.tradeDate)
        self.CME_PAGE_FILE_OPT = '{0}_options_{1}{2}_{3}.pkl'.format(self.options_sym.lower(), 
                                    self.expiry_month, self.expiry_year, self.tradeDate)
        
        self.CME_PAGE_FILE_OPT = '{0}_options_{1}{2}_{3}.pkl'.format(self.options_sym.lower(),                                                         self.expiry_month,
                                                            self.expiry_year, self.tradeDate)
        
        
        self.cme_plot_opt = '{0}_options_{1}{2}_{3}.png'.format(self.options_sym.lower(),                                                         self.expiry_month,
                                                            self.expiry_year, self.tradeDate)
        
        self.CME_PAGE_FILE_FUT = '{0}_futures_{1}.pkl'.format(self.futures_sym.lower(), self.tradeDate)
        
        self.expiry_cal = {'NG': 'expiry_cal_ng.xlsx', 'CL':'expiry_cal_cl.xlsx' }
        
        self.mon_symbols = {'JAN': 'F', 'FEB': 'G', 'MAR': 'H', 'APR': 'J', 'MAY': 'K', 'JUN': 'M', 'JLY': 'N',
                            'AUG': 'Q','SEP': 'U', 'OCT': 'V', 'NOV': 'X', 'DEC': 'Z'}
        
        self.mon_num = {'F': '01', 'G': '02', 'H': '03', 'J': '04', 'K': '05', 'M': '06' ,'N': '07',
                        'Q': '08', 'U': '09', 'V': '10', 'X': '11', 'Z': '12'}

        # CME product symbols    
        self.options = {'ON': '191', 'LN':'1352', 'LO': '190'}   
        self.cme_product = {'CL': ['energy', 'crude_oil', 'light-sweet-crude'],
                            'NG': ['energy', 'natural-gas', 'natural-gas']
                            }
        
        # dictionary for css selection
        self.css_month = {'0': {'M':'1352-M0','N':'1352-N0', 'Q':'1352-Q0', 'U': '1352-U0', 'V':'1352-V0', 'X':'1352-X0', 
                    'Z':'1352-Z0'}, '1':{'F':'1352-F1', 'G':'1352-G1', 'H': '1352-H1', 'J':'1352-J1', 
                    'Z':'1352-Z1'}}

        # parse tradeDate
        self.parsed = datetime.strptime(tradeDate, '%Y-%m-%d')
        self.parsed = self.parsed.strftime('%Y-%m-%d').split('-')
        
        # css dropdown date
        self.css_date = "{0}/{1}/{2}".format(self.parsed[1], self.parsed[2], self.parsed[0])
        
        # build url for CME NYMEX products
        self.url_opt_ng = "https://www.cmegroup.com/trading/energy/natural-gas/natural-gas_quotes_settlements_options.html#" \
                            "optionProductId=1352&optionExpiration=1352-{0}{1}&tradeDate={2}%2F{3}%2F{4}".format(self.expiry_month, 
                                        self.expiry_year, self.parsed[1], self.parsed[2], self.parsed[0] )

        self.url_opt_ng = "https://www.cmegroup.com/trading/energy/natural-gas/natural-gas_quotes_settlements_options.html#" \
                            "optionProductId=1352&optionExpiration=1352-N0&tradeDate=05%2F21%2F2020"    
        
        logging.info(self.url_opt_ng)
        self.url_fut_ng = "https://www.cmegroup.com/trading/energy/natural-gas/natural-gas_quotes_settlements_futures.html?" \
        "optionProductId=1352&optionExpiration=1352-{0}{1}#tradeDate={2}%2F{3}%2F{4}".format(self.expiry_month, 
                                        self.expiry_year, self.parsed[1], self.parsed[2], self.parsed[0] )
        logging.info(self.url_fut_ng)
        
        self.xpath_fut = '//*[@id="settlementsFuturesProductTable"]'
        self.xpath_opt = '//*[@id="settlementsOptionsProductTable"]'
        
        self.months = {'JAN':'F', 'FEB':'G', 'MAR':'H', 'APR':'J', 'MAY':'K', 'JUN':'M', 'JLY':'N',
                      'AUG':'Q', 'SEP':'U', 'OCT':'V', 'NOV':'X', 'DEC':'Z'}
    
    def choose_url(self, futures_sym):

        if futures_sym == 'NG':
            url_opt = self.url_opt_ng
            url_fut = self.url_fut_ng
        elif futures_sym == 'CL':
            url_opt = self.url_opt_cl
            url_fut = self.url_fut_cl
        else:
            url_opt = None
            url_fut = None
            logging.info("Wrong futures symbol chosen.")

        return url_opt, url_fut
    
    def splinter_driver(self):
        self._splbrowser = Browser('firefox')
        
    def selenium_driver(self):
        self._selbrowser = webdriver.Firefox()
        
        
    def splinter_scrape_fut(self, url):
        """Run splinter to fetch url text data table."""
        
        browser = Browser('firefox')
        browser.visit(url)
        
        browser.click_link_by_id('cmeTradeDate')
        browser.find_by_value(self.css_date).first.click()
        
        time.sleep(2)
        table = browser.find_by_xpath(self.xpath_fut, wait_time=5).first.text
        table_list = table.split('\n')  
        
        browser.quit()
        
        return table_list
    
        
    def splinter_scrape_opt(self, url):
        """Run splinter to fetch url text data table. Select dropdown menu."""
        
        # selection choices can be expanded
        mon = {'M': '1', 'N':'2', 'Q':'3', 'U':'4', 'V':'5', 'X':'6'}
        css_month = self.css_month
        
        
        selected = css_month[self.expiry_year][self.expiry_month]
        
        css_sel = '#deliveryMonth1 > option:nth-child({0})'.format(selected)       
        
        browser = Browser('firefox')
        browser.visit(url)
        
        browser.find_by_name("optionExpiration") 
        browser.find_by_css(css_sel).first.click()  
        
        browser.click_link_by_id('cmeTradeDate')
        browser.find_by_value(self.css_date, wait_time=5).first.click()
        
        xpath_opt = '//*[@id="settlementsOptionsProductTable"]'
        css = '#settlementsOptionsProductTable'
        
        time.sleep(2)
        table = browser.find_by_css(css).first.text
        
        # parse table
        table_list = table.split('\n')    
        # print(table_list)
        
        browser.quit()
        
        return table_list
    
    
    def selenium_scrape_opt(self, url):
        """Using Selenium library to scrape page."""
        
        browser = webdriver.Firefox()
        browser.get(url)
        
        xpath_opt = self.xpath_opt
        dt = self.css_date
        css_month = self.css_month
    
        selected_month = css_month[self.expiry_year][self.expiry_month]
        
        
        elem_select = Select(browser.find_element_by_id('deliveryMonth1'))  
        elem_select.select_by_value(selected_month)
        elem_select = Select(browser.find_element_by_id('cmeTradeDate')) 
        elem_select.select_by_value(dt) 
        elem_select = Select(browser.find_element_by_id('cmeTradeDate'))
        elem_select.select_by_value(dt)
        
        # sleep function needed to push page to send correct prices for tradeDate
        time.sleep(5)  
        table  = browser.find_element_by_xpath(xpath_opt).text
        
        # parse table
        table_list = table.split('\n')    
        
        browser.quit()
        
        return table_list
        
    
    def scrape_cme_futures_page(self):
        """Scrape futures prices from CME settlements web page."""

        text_list = self.splinter_scrape_fut(self.url_fut_ng)

        headers = text_list[0]
        headers = headers.split(' ')[:7]
        cols = [s.upper() for s in headers]

        cols.append('EST_VOLUME')
        cols.append('OPEN_INTEREST')

        fut = text_list[3:]
        fut.pop()
        num_rows = len(fut)
        fut_list = []

        # iterate through the list of futures and create contract month: NGQ20,
        for i in range(num_rows):
            s = fut[i].replace(fut[i][:6], '{0}{1}{2}'.format(self.futures_sym, self.mon_symbols[fut[i][:3]],
                                                              (fut[i][:6])[4:]))
            s = s.replace('-', '0').replace(',', '').replace('A', '').replace('B', '').replace('UNCH', '0').split(' ')
            tmp = [float(elem) for elem in s if elem != s[0]]
            tmp.insert(0, s[0])
            fut_list.append(tmp)

        # finally create pandas DataFrame
        fut_df = pd.DataFrame(fut_list, columns=cols)
        fut_df.index.name = "idx"
        fut_df.rename(columns={"MONTH": "CONTRACT"}, inplace=True)
        fut_df = fut_df[['CONTRACT', 'OPEN', 'HIGH', 'LOW', 'LAST', 'SETTLE', 'EST_VOLUME', 
                         'OPEN_INTEREST']]

        return fut_df

    
    def scrape_cme_options_page(self):
        """Scrape CME website for Option prices using Selenium library."""

        # Using splinter library instead of selenium
        #text_list = self.splinter_scrape_opt(self.url_opt_ng)
        text_list = self.selenium_scrape_opt(self.url_opt_ng)

        headers = text_list[0]
        # parse tradeDate from options table scraped
        dt = parse(text_list[1][14:]).date().strftime('%Y-%m-%d') 
        
        headers = headers.split(' ')[:8]
        cols = [s.upper() for s in headers]
        cols.append('EST_VOLUME')  # add back Volume and Open interest since it got taken out
        cols.append('OPEN_INTEREST')

        opt = text_list[3:]
        opt.pop()  # pop totals row

        num_rows = len(opt)  # number of rows in list options
        opt_list = []  # to create list of list of options
            
        for i in range(num_rows):
            xopt = opt[i].split(' ')
            t = [float(a.replace('B', '').replace('A', '').replace('-', '0').replace('UNCH', '0').replace(',', '')) for
                 a in xopt if a != 'Call' if a != 'Put']
            t.insert(1, xopt[1])
            opt_list.append(t)

        # create dataframe of option prices
        options_df = pd.DataFrame(opt_list, columns=cols)
        options_df['OPT_TRADEDATE'] = dt
        options_df['CONTRACT'] = self.contract
        options_df['OPT_SYM'] = self.options_sym
        options_df = options_df[options_df['SETTLE'] >= 0.005]
        options_df = options_df[['STRIKE', 'TYPE', 'CONTRACT', 'OPT_SYM', 'OPEN', 'HIGH', 'LOW', 'LAST',
                                 'OPT_TRADEDATE', 'SETTLE', 'EST_VOLUME', 'OPEN_INTEREST']]

        return options_df
    
        # Black Scholes model
        
    def icnd(self, d):
        """Moro's 1994 algorithm for the Inverse Cumulative Normal Distribuiton"""
        
        a = np.array([2.50662823884, -18.61500062529, 41.39119773534, -25.44106049637])
        b = np.array([-8.47351093090, 23.08336743743, -21.06224101826, 3.13082909833])
        c = np.array([0.3374754822726147, 0.9761690190917186, 0.1607979714918209, 0.0276438810333863, 
                        0.0038405729373609, 0.0003951896511919, 0.0000321767881768, 0.0000002888167364,
                        0.0000003960315187])
        
        x = d - 0.5
        if np.abs(x) < 0.92:
            r = x**2
            r = x * ((( a[3] * r + a[2])*r  + a[1]) * r + a[0])/ ((((b[3] * r + b[2]) * r + b[1]) * r + b[0]) * r + 1)
            return r
        
        r = d
        if x >= 0:
            r = 1 - d
            r = np.log(-np.log(r))
            r = c[0] + r * (c[1] + r * (c[2] + r * (c[3] + r + (c[4] + r * (c[5] + r * (c[6] + r * (c[7] + r * c[8])))))))
        elif d < 0 :
            r = -r
            return r
        
    def strike_from_delta(self, cpflag, S, T, r, b, v, d):
        """Strike from delta. Haug-2007, p.31"""
        
        exprt = np.exp((r - b) * T)
        if cpflag == 'Call':
            return S * np.exp(-self.icnd(d * exprt) * v * np.sqrt(T) + (b + v**2 / 2.) * T)
        else:
            return S * np.exp(self.icnd(-d * exprt) * v * np.sqrt(T) + (b + v**2 / 2.) * T)

    
    def GBlackScholes(self, cpflag, S, X, T, r, b, V):
        '''Generalized Black76 European options model for Futures.'''
        Gd1 = (np.log(S / X) + (b + V ** 2 / 2) * T) / (V * np.sqrt(T))
        
        Gd2 = Gd1 - V * np.sqrt(T)
        if cpflag == "Call":
            return S * np.exp((b - r) * T) * ss.norm.cdf(Gd1) - X * np.exp(-r * T) * ss.norm.cdf(Gd2)
        else:
            return X * np.exp(-r * T) * ss.norm.cdf(-Gd2) - S * np.exp((b - r) * T) * ss.norm.cdf(-Gd1)

    def delta(self, cpflag, S, X, T, r, b, V):
        """Generalized Black_Scholes delta for Options on futures."""
        
        # logging.info(S, X, T, r, b, V)
        
        Gd1 = (np.log(S / X) + (b + V ** 2 / 2) * T) / (V * np.sqrt(T))
        
        if cpflag == "Call":
            gdelta = np.exp((b - r) * T) * ss.norm.cdf(Gd1)
        else:
            gdelta = -np.exp((b - r) * T) * ss.norm.cdf(-Gd1)
        
        return gdelta

    def gvega(self, S, X, T, r, b, V):
        """Generalized Vega for Black-Scholes."""

        Vd1 = (np.log(S / X) + (b + V ** 2 / 2) * T) / (V * np.sqrt(T))

        return S * np.exp((b - r) * T) * ss.norm.pdf(Vd1) * np.sqrt(T)

    def GImpliedVolatility(self, cpflag, S, X, T, r, b, cm, epsilon):
        """Calculate implied volatility using Newton-Raphson method with initial vol seed."""

        vi = np.sqrt(abs(np.log(S / X) + r * T) * 2 / T)  # initial vol
        
        # print(cpflag, S, X, T, r, b, cm, epsilon)
        ci = self.GBlackScholes(cpflag, S, X, T, r, b, vi)
        vegai = self.gvega(S, X, T, r, b, vi)
        min_diff = abs(cm - ci)

        while (abs(cm - ci) >= epsilon) & (abs(cm - ci) <= min_diff):
            vi = vi - (ci - cm) / vegai
            ci = self.GBlackScholes(cpflag, S, X, T, r, b, vi)
            vegai = self.gvega(S, X, T, r, b, vi)
            min_diff = abs(cm - ci)
        if abs(cm - ci) < epsilon:
            vi = vi
        else:
            vi = 'NA'

        return vi

    def attach_underlying_to_options_df(self, future_prices, option_prices):
        """Attach underlying price to options dataframe."""

        # attach underlying to options
        future_prices.index = future_prices['CONTRACT']
        
        #xlwings.view(future_prices)
        u = [future_prices['SETTLE'].loc[row] for row in future_prices.index for elem in option_prices['CONTRACT'] if
             row == elem]
        option_prices['UNDERLYING'] = u

        if ( self.options_sym == 'LN' or self.options_sym == 'ON'):
            option_prices['STRIKE'] = option_prices['STRIKE'] / 1000.
        else:
            option_prices['STRIKE'] = option_prices['STRIKE'] / 100.
        
        
        return option_prices

    def attach_expiry_date_options_df(self, option_prices):
        """Attach expiry date to options dataframe, use futures symbol to find file."""

        expiry_file = pd.read_excel(self.expiry_cal[self.futures_sym], header=0, index_col=0)
        
        # attach expiry date to options dataframe
        option_prices['EXPIRY'] = [expiry_file.loc[row, 'Settlement'] for row in expiry_file.index for contract in
                            option_prices['CONTRACT'] if row == contract]

        option_prices['EXPIRY'] = pd.to_datetime(option_prices['EXPIRY'])
        
        
        tradeDate = pd.to_datetime(option_prices['FUT_TRADEDATE'].iloc[0])
        option_prices['DTE'] = (option_prices["EXPIRY"] - tradeDate).dt.days

        return option_prices  
         
    def get_prices_cme_web_page(self):
        """Get prices from either a stored pickle file or CME ftp site."""

        # get stored prices from Pickle file or Database
        try:
            future_prices = pd.read_pickle("{0}/{1}".format(ROOT_DIR, self.CME_PAGE_FILE_FUT))
        except FileNotFoundError:
            future_prices = self.scrape_cme_futures_page()
            future_prices.to_pickle("{0}/{1}".format(ROOT_DIR, self.CME_PAGE_FILE_FUT))

        try:
            option_prices = pd.read_pickle("{0}/{1}".format(ROOT_DIR, self.CME_PAGE_FILE_OPT))
        except FileNotFoundError:
            option_prices = self.scrape_cme_options_page()
            option_prices.to_pickle("{0}/{1}".format(ROOT_DIR, self.CME_PAGE_FILE_OPT))

        # try:
        #    libor_rates = pd.read_pickle("{0}/data/{1}".format(ROOT_DIR, self.LIBOR_FILE))
        # except FileNotFoundError:
        #    libor_rates = self.get_libor_rates()
        #    libor_rates.to_pickle("{0}/data/{1}".format(ROOT_DIR, self.LIBOR_FILE))
        
        #xlwings(option_prices)
        # attach underlying prices as well as option expiry dates, add tradeDate
        option_prices['FUT_TRADEDATE'] = self.tradeDate
        option_prices = self.attach_underlying_to_options_df(future_prices, option_prices)
        option_prices = self.attach_expiry_date_options_df(option_prices)
    
        
        return option_prices, future_prices
    
    def plot_vol_surface(self, x, y, put_call, contract):
        """Plot surface using Ordinary Least Square Fit."""

        df = pd.DataFrame(columns=['x', 'y'])
        df['x'] = x
        df['y'] = y
        degree = 3

        try:
            weights = np.polyfit(x, y, degree)
            model = np.poly1d(weights)
            results = smf.ols(formula='y ~ model(x)', data=df).fit()
            prstd, iv_l, iv_u = wls_prediction_std(results)
            fig, ax = plt.subplots(figsize=(12, 8))
            plt.title(
            "Implied Vol for NG European Options = {0}, moneyness plot= log(K/F): tradeDate: {1}".format(contract,
                                                                                            self.tradeDate))
            ax.plot(x, y, 'o', label="{0} Implied Vol".format(put_call))
            ax.plot(x, results.fittedvalues, 'r--.', label="OLS")
            ax.plot(x, iv_u, 'r--')
            ax.plot(x, iv_l, 'r--')
            ax.legend(loc='best')
            plt.xlabel("Moneyness: log(K/F)")
            plt.ylabel("Implied Vol.")
            plt.axvline(0, color='k')
            plt.savefig("{0}/{1}".format(ROOT_DIR, self.cme_plot))
            plt.show()

        except ValueError:
            logging.info("ValueError!")
            
    def parse_user_input(self):
        """Get user input for contract name."""
        
        contract = input("Enter Natural Gas Contract (e.g. {0}Q20): ".format(self.futures_sym)).upper()
        
        while(len(contract) != 5):
            logging.info("Wrong number of characters in contract name!")
            contract = input("Enter Natural Gas Contract (e.g. {0}Q20): ".format(self.futures_sym)).upper()
            
        try:
            self.futures_sym = contract[:2]
            self.expiry_month = contract[2]
            self.expiry_year = contract[4]
            self.CME_PAGE_FILE_OPT = '{0}_options_{1}{2}_{3}.pkl'.format(self.options_sym.lower(),                                                         self.expiry_month,
                                                                     self.expiry_year, self.tradeDate)
        except:
            logging.info("Wrong input")
            contract = input("Enter Natural Gas Contract (e.g. {0}Q20): ".format(self.futures_sym)).upper()
        return contract
    
    
    def vol_surface(self, put_call='Call'):
        """Generate vol surface using Moneyness: log(Strike/Underlying)"""
        
        contract = self.contract
        
        option_prices, future_prices = self.get_prices_cme_web_page()
        
        # filter for moneyness

        option_prices['moneyness'] = np.log(option_prices['STRIKE'] / option_prices['UNDERLYING'])
        logging.info("Calculated moneyness")
    
        # create option strike intervals
        max_strike = np.max(option_prices['STRIKE'])
        min_strike = np.min(option_prices['STRIKE'])
        if self.futures_sym == 'NG':
            strike_interval = np.arange(min_strike, max_strike, 0.05).round(2).tolist()
            option_prices = option_prices[option_prices['STRIKE'].isin(strike_interval)]
        elif self.futures_sym == 'CL':
            strike_interval = np.arange(min_strike, max_strike, 0.5).round(2).tolist()
            option_prices = option_prices[option_prices['STRIKE'].isin(strike_interval)]
        
        # print(strike_interval)
        option_prices = option_prices[option_prices['STRIKE'].isin(strike_interval)]
        # xlwings.view(dg)
        
        option_prices = option_prices[option_prices['moneyness'] <= 0.8]
        df = option_prices[option_prices['moneyness'] >= -0.8]

        # low interest rate constant for now,  not material in this low rate environment
        rate = self.rate  
        epsilon = self.epsilon
        b = self.carry  

        n = len(df)
        vols = []
        delta = []
        # iterate through each option type and price to generate Implied Vol
        for i in range(n):
            cpflag = df['TYPE'].iloc[i]
            F = df['UNDERLYING'].iloc[i]
            K = df['STRIKE'].iloc[i]
            T = float(df['DTE'].iloc[i]) / self.basis
            r = rate;
            cm = df['SETTLE'].iloc[i]
            b = 0  # for futures

            try:
                iv = self.GImpliedVolatility(cpflag, F, K, T, r, b, cm, epsilon)
                vols.append(iv)        
            except OverflowError:
                logging.info("Unable to converge to an implied vol.")
                vols.append(0.)
                
            try:
                dlt = self.delta(cpflag, F, K, T, r, b, iv)
                delta.append(dlt)
            except ValueError:
                logging.info("Unable to calculate delta.")
                delta.append(0.)

        df['implied_vol'] = vols
        df['delta'] = delta

        df = df[df['TYPE'] == put_call]
        df = df[df['implied_vol']!='NA']
        x = [float(elem) for elem in df['moneyness']]
        y = [float(elem) for elem in df['implied_vol']]
        
        self.plot_vol_surface(x, y, put_call, contract)

        # save vol surface to pickle file
        df.to_pickle("{0}/{1}".format(ROOT_DIR, self.SRC_VOL_FILE))

        return df
    
        

In [3]:
if __name__ == '__main__':
    
    month = 'V'
    year = '0'
    dt = '2020-09-16'
    put_call = 'Call'
    d = GetData(tradeDate=dt, options_sym='LN', expiry_month=month, year=year)
    
    # opt_prices, fut_prices = d.get_prices_cme_web_page()
    opt_df = d.vol_surface(put_call=put_call)
    
    delta = d.delta('Call', 105, 100, 0.5, 0.1, 0, 0.36)
    
    normal = d.icnd(0.8)

INFO:root:https://www.cmegroup.com/trading/energy/natural-gas/natural-gas_quotes_settlements_options.html#optionProductId=1352&optionExpiration=1352-N0&tradeDate=05%2F21%2F2020
INFO:root:https://www.cmegroup.com/trading/energy/natural-gas/natural-gas_quotes_settlements_futures.html?optionProductId=1352&optionExpiration=1352-V0#tradeDate=09%2F16%2F2020


WebDriverException: Message: 'geckodriver' executable needs to be in PATH. 


In [5]:
for _ in range(20):
    normal = d.icnd(np.random.uniform())
    print(normal)

1.610809597491511
0.1676066542345985
-1.092795351497305
0.10848935488258024
-0.7591084979631484
0.6508862493459755
0.5791630203466231
0.537801775422384
-1.547562968906082
-0.5913670741358817
-0.03892177937891951
-2.075678776251574
0.2782614092333403
-0.058135895704890486
-0.3272468504069808
2.003498386003279
-0.688468239076934
-0.20712104172085677
2.119440859450398
0.7045933755537694


In [105]:
X = d.strike_from_delta('Call', 2.792, 0.512329, 0.0035, 0, 0.579860761, 0.102290428)
X

5.150000814358122

In [102]:
np.arange(1,4,0.05).round(4).tolist()

[1.0,
 1.05,
 1.1,
 1.15,
 1.2,
 1.25,
 1.3,
 1.35,
 1.4,
 1.45,
 1.5,
 1.55,
 1.6,
 1.65,
 1.7,
 1.75,
 1.8,
 1.85,
 1.9,
 1.95,
 2.0,
 2.05,
 2.1,
 2.15,
 2.2,
 2.25,
 2.3,
 2.35,
 2.4,
 2.45,
 2.5,
 2.55,
 2.6,
 2.65,
 2.7,
 2.75,
 2.8,
 2.85,
 2.9,
 2.95,
 3.0,
 3.05,
 3.1,
 3.15,
 3.2,
 3.25,
 3.3,
 3.35,
 3.4,
 3.45,
 3.5,
 3.55,
 3.6,
 3.65,
 3.7,
 3.75,
 3.8,
 3.85,
 3.9,
 3.95]

In [20]:
a = np.array([2.50662823884, -18.61500062529, 41.39119773534, -25.44106049637])

In [22]:
a[1]

-18.61500062529

In [64]:
df = pd.read_pickle('ng_futures_2020-05-20.pkl')

In [67]:
dg = pd.read_pickle('ln_options_N0_2020-05-20.pkl')

In [68]:
dg

,STRIKE,TYPE,CONTRACT,OPT_SYM,OPEN,HIGH,LOW,LAST,SETTLE,EST_VOLUME,OPEN_INTEREST
0,800.0,Call,NGN20,LN,0.0,0.000,1.055,1.055,0.0,0.0,2.0
1,1000.0,Call,NGN20,LN,0.0,0.000,0.856,0.857,0.0,0.0,413.0
2,1150.0,Call,NGN20,LN,0.0,0.000,0.709,0.709,0.0,0.0,8.0
3,1200.0,Call,NGN20,LN,0.0,0.000,0.660,0.660,0.0,0.0,6.0
4,1250.0,Call,NGN20,LN,0.0,0.000,0.613,0.613,0.0,0.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...
171,14000.0,Put,NGN20,LN,0.0,12.111,0.000,12.111,0.0,0.0,60.0
172,14250.0,Put,NGN20,LN,0.0,12.360,0.000,12.360,0.0,0.0,109.0
173,14500.0,Put,NGN20,LN,0.0,12.610,0.000,12.610,0.0,0.0,182.0
174,14750.0,Put,NGN20,LN,0.0,12.859,0.000,12.859,0.0,0.0,159.0


In [139]:
%pip install mibian --user

Note: you may need to restart the kernel to use updated packages.


In [116]:
opt_df

,STRIKE,TYPE,CONTRACT,OPT_SYM,OPEN,HIGH,LOW,LAST,OPT_TRADEDATE,SETTLE,EST_VOLUME,OPEN_INTEREST,FUT_TRADEDATE,UNDERLYING,EXPIRY,DTE,moneyness,implied_vol,delta
0,1.00,Call,NGU20,LN,0.000,1.074,0.000,1.065,2020-05-26,1.0784,0.0,1100.0,2020-05-26,2.069,2020-08-26,92,-0.727065,0.847196,0.971843
1,1.10,Call,NGU20,LN,0.000,0.977,0.000,0.969,2020-05-26,0.9822,0.0,0.0,2020-05-26,2.069,2020-08-26,92,-0.631755,0.794268,0.961912
2,1.25,Call,NGU20,LN,0.000,0.838,0.000,0.831,2020-05-26,0.8417,0.0,0.0,2020-05-26,2.069,2020-08-26,92,-0.503922,0.737535,0.938127
3,1.35,Call,NGU20,LN,0.000,0.748,0.000,0.741,2020-05-26,0.7520,0.0,0.0,2020-05-26,2.069,2020-08-26,92,-0.426961,0.712850,0.914154
4,1.45,Call,NGU20,LN,0.000,0.662,0.000,0.655,2020-05-26,0.6659,0.0,6.0,2020-05-26,2.069,2020-08-26,92,-0.355502,0.695311,0.882773
5,1.50,Call,NGU20,LN,0.000,0.621,0.000,0.615,2020-05-26,0.6244,0.0,0.0,2020-05-26,2.069,2020-08-26,92,-0.321600,0.684358,0.865265
6,1.60,Call,NGU20,LN,0.000,0.542,0.000,0.537,2020-05-26,0.5453,0.0,0.0,2020-05-26,2.069,2020-08-26,92,-0.257062,0.667043,0.824390
7,1.70,Call,NGU20,LN,0.000,0.469,0.000,0.464,2020-05-26,0.4715,0.0,0.0,2020-05-26,2.069,2020-08-26,92,-0.196437,0.652468,0.776721
8,1.75,Call,NGU20,LN,0.000,0.434,0.000,0.429,2020-05-26,0.4368,0.0,1.0,2020-05-26,2.069,2020-08-26,92,-0.167450,0.646001,0.750599
9,1.80,Call,NGU20,LN,0.000,0.401,0.349,0.396,2020-05-26,0.4036,0.0,1.0,2020-05-26,2.069,2020-08-26,92,-0.139279,0.639847,0.723169


In [124]:
pwd

'/Users/luismolina/OneDrive/Hartree_Project/implied_volatility'

In [14]:
pwd

'/Users/luismolina/OneDrive/Hartree_Project/implied_volatility'